In [1]:
!pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = gemini_api_key

In [3]:
!pip install langchain-google-genai

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

/Users/macintosh/TA-DOCUMENT/StudyZone/FPT_WORK/IVY Training/IVY DEV/DailyReport/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Message passing

In [11]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)
print(ai_msg.content)

I just said "I love programming" in French.


### Chat history

In [8]:
!pip install langgraph

In [42]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [43]:
app.invoke(
    {"messages": [HumanMessage(content="Translate to French: I love programming.")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='Translate to French: I love programming.', additional_kwargs={}, response_metadata={}, id='2b241221-f730-47a5-a6a1-1f8f1930f905'),
  AIMessage(content="The most straightforward translation is:  **J'aime la programmation.**", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0aea9680-c271-4ef3-aec3-d398921dc0df-0', usage_metadata={'input_tokens': 25, 'output_tokens': 15, 'total_tokens': 40, 'input_token_details': {'cache_read': 0}})]}

In [44]:
app.invoke(
    {"messages": [HumanMessage(content="What did I just ask you?")]},
    config={"configurable": {"thread_id": "2"}},
)

{'messages': [HumanMessage(content='What did I just ask you?', additional_kwargs={}, response_metadata={}, id='dc6bcc0a-88e3-44be-8add-b0a0de1f1772'),
  AIMessage(content='You asked me: "What did I just ask you?"', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-5def1070-c323-4d5a-9241-f80fa6597a83-0', usage_metadata={'input_tokens': 24, 'output_tokens': 13, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})]}

### Trimming Message

In [45]:
demo_ephemeral_chat_history = [
    HumanMessage(content="Hey there! I'm Nemo."),
    AIMessage(content="Hello!"),
    HumanMessage(content="How are you today?"),
    AIMessage(content="Fine thanks!"),
]

app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage(content="What's my name?")]
    },
    config={"configurable": {"thread_id": "2"}},
)

{'messages': [HumanMessage(content='What did I just ask you?', additional_kwargs={}, response_metadata={}, id='dc6bcc0a-88e3-44be-8add-b0a0de1f1772'),
  AIMessage(content='You asked me: "What did I just ask you?"', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-5def1070-c323-4d5a-9241-f80fa6597a83-0', usage_metadata={'input_tokens': 24, 'output_tokens': 13, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}, id='b57b58a8-1197-450b-a889-b9bcf3688c8b'),
  AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='2150d04c-3fd6-4db9-8f22-c7b58c20c693'),
  HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}, id='00f611e3-7329-4b7c-a607-60fca3f9991a'),
  AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}

In [46]:
from langchain_core.messages import trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define trimmer
# count each message as 1 "token" (token_counter=len) and keep only the last two messages
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + trimmed_messages
    response = llm.invoke(messages)
    print("trim_messages", trimmed_messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [47]:
app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage(content="What is my name?")]
    },
    config={"configurable": {"thread_id": "2"}},
)

trim_messages [AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}, id='b09aaf58-3040-49f2-8fce-8ee8e7b6b972'), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='d60874e0-75c7-4624-840b-65e88f3ed0fb')]


{'messages': [HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}, id='b57b58a8-1197-450b-a889-b9bcf3688c8b'),
  AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='2150d04c-3fd6-4db9-8f22-c7b58c20c693'),
  HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}, id='00f611e3-7329-4b7c-a607-60fca3f9991a'),
  AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}, id='b09aaf58-3040-49f2-8fce-8ee8e7b6b972'),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='d60874e0-75c7-4624-840b-65e88f3ed0fb'),
  AIMessage(content='I do not know your name.  I have no access to personal information about you unless you explicitly provide it to me.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-deba2567-d553-41f5-bb0c-6127d362b9c0-0', usage_m

### Summary memory

In [48]:
demo_ephemeral_chat_history = [
    HumanMessage(content="Hey there! I'm Nemo."),
    AIMessage(content="Hello!"),
    HumanMessage(content="How are you today?"),
    AIMessage(content="Fine thanks!"),
]

In [49]:
from langchain_core.messages import HumanMessage, RemoveMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability. "
        "The provided chat history includes a summary of the earlier conversation."
    )
    system_message = SystemMessage(content=system_prompt)
    message_history = state["messages"][:-1]  # exclude the most recent user input
    # Summarize the messages if the chat history reaches a certain size
    if len(message_history) >= 4:
        last_human_message = state["messages"][-1]
        # Invoke the model to generate conversation summary
        summary_prompt = (
            "Distill the above chat messages into a single summary message. "
            "Include as many specific details as you can."
        )
        summary_message = llm.invoke(
            message_history + [HumanMessage(content=summary_prompt)]
        )

        # Delete messages that we no longer want to show up
        delete_messages = [RemoveMessage(id=m.id) for m in state["messages"]]
        # Re-add user message
        human_message = HumanMessage(content=last_human_message.content)
        # Call the model with summary & response
        response = llm.invoke([system_message, summary_message, human_message])
        message_updates = [summary_message, human_message, response] + delete_messages
    else:
        message_updates = llm.invoke([system_message] + state["messages"])

    return {"messages": message_updates}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [50]:
app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage("What did I say my name was?")]
    },
    config={"configurable": {"thread_id": "4"}},
)

{'messages': [AIMessage(content='Nemo initiated a conversation by saying "Hey there! I\'m Nemo."  I responded with "Hello!" and then Nemo asked "How are you today?", to which I replied "Fine thanks!".', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f6151420-21da-4e94-a1f6-3df243937f89-0', usage_metadata={'input_tokens': 44, 'output_tokens': 41, 'total_tokens': 85, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content='What did I say my name was?', additional_kwargs={}, response_metadata={}, id='3ba04d91-9bcf-49fb-ab8c-936a60643609'),
  AIMessage(content='You said your name was Nemo.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-7b2a9782-9626-41c4-9053-d4b1eb633c32-0', usage_metadata={'input_tokens': 78, 'output_tokens': 8, 'total_tokens': 86, 'input